In [20]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

In [21]:
driver = webdriver.Chrome(r"C:/chromedriver_win32/chromedriver.exe")

In [22]:
player_summary_df = pd.DataFrame(columns=["R", "player", "height", "weight", "apps",
                                          'mins','goals','Assists','Yel','Red',
                                          'SpG','PS%','AerialsWon','MotM','Rating'])

In [23]:
#Get Teams Names and ids
def getTeams():
    #Create URL
    driver.get('https://www.whoscored.com/Regions/252/Tournaments/2/')
    #Go to the wanted data table
    base = driver.find_element_by_id('standings-16368-content')
    #Get table rows
    rows =base.find_elements_by_tag_name('tr')
    teams = []
    for item in range(0,len(rows)):
        #Go to specific location to find the team name
        data = rows[item].find_elements_by_tag_name('td')
        teams.append(data[1].text)
    links = []
    for i in range(0,len(rows)):
        #Get the team ID
        links.append(rows[i].find_elements_by_class_name('team-link')[0].get_attribute('href'))
    team_ids = []
    #With regular expressions module, extract the id from team's link.
    for i in range(0,len(links)):
        team_ids.append(re.search(r'\d+', links[i]).group())
    team_ids = list(map(int, team_ids))
    for i in range(0,len(teams)):
        #Replace spaces in team name
        if ' ' in teams[i]:
            teams[i] = teams[i].replace(' ','-')
    #Construct a dict of team names and ids
    teams = dict(zip(teams, team_ids))
    return teams
teams = getTeams()

In [ ]:
"""Get Summary stats Stats Function"""
players_names = []
players_ids = []
def getPlayerSummary(teams):
    values =[]
    for i in teams.values(): 
        driver.get('https://www.whoscored.com/Teams/'+str(i))
        driver.implicitly_wait(40)
        base = driver.find_element_by_id('top-player-stats-summary-grid')
        table = base.find_element_by_id('player-table-statistics-body')
        tableRows = table.find_elements_by_tag_name('tr')
        for item in range(1,len(tableRows)):
                link = tableRows[item].find_elements_by_class_name('player-link')[0].get_attribute('href')
                players_ids.append(re.search(r'\d+',link).group())
                row     = tableRows[item].find_elements_by_tag_name('td')
                players_names.append(row[1].text)
                values.append([row[i].text for i in range(1,len(row))])
    list_of_players = list(zip(players_names, players_ids))
    players = pd.DataFrame(list_of_players, columns = ['Name', 'ID'])
    dfOut = pd.DataFrame(values,columns=["R", "Name", "height", "weight", "apps",'mins','goals','Assists','Yel','Red',
                                         'SpG','PS%','AerialsWon','MotM','Rating'])
    dfOut = pd.merge(dfOut,players,on='Name')
    return dfOut
players_summary = getPlayerSummary(teams)

In [24]:
driver.get('https://www.whoscored.com/Teams/32/Show/England-Manchester-United')
driver.implicitly_wait(40)
base = driver.find_element_by_id('top-player-stats-summary-grid')
table = base.find_element_by_id('player-table-statistics-body')
tableRows = table.find_elements_by_tag_name('tr')

In [26]:
tableRows[0].find_elements_by_class_name('player-link')[0].get_attribute('href')

'https://www.whoscored.com/Players/97752/Show/Paul-Pogba'

In [9]:
"""Get Defensive Stats Function"""

def getDefensiveStats(teams):
    #Create empty list for future data
    values=[]
    #set Url
    driver.get('https://www.whoscored.com/Teams/32/Show/England-Manchester-United')
    #Wait for the page to load.
    driver.implicitly_wait(40)
    #Get the right table by clicking the defensive tab on the data table.
    driver.find_elements_by_id("team-squad-stats-options")[0].find_elements_by_tag_name('li')[1].click()
    base = driver.find_elements_by_id('statistics-table-defensive')
    tableRows = base[0].find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row     = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values)
    #return the data table  
    return df
data = getDefensiveStats(teams)

In [18]:
"""Get Attacking Stats Function"""

def getAttackingStats(teams):
    #Create empty list for future data
    values=[]
    #set Url
    driver.get('https://www.whoscored.com/Teams/32/Show/England-Manchester-United')
    #Wait for the page to load.
    driver.implicitly_wait(40)
    #Get the right table by clicking the defensive tab on the data table.
    driver.find_elements_by_id("team-squad-stats-options")[0].find_elements_by_tag_name('li')[2].click()
    base = driver.find_elements_by_id('statistics-table-offensive')
    tableRows = base[0].find_elements_by_tag_name('tr')
    for item in range(1,len(tableRows)):
        row     = tableRows[item].find_elements_by_tag_name('td')
        values.append([row[i].text for i in range(1,len(row))])
    df = pd.DataFrame(values)
    #return the data table  
    return df
data = getAttackingStats(teams)